# About this Notebook:

This notebook detailed the exploratory data analysis on the Open Images Classes. So we can prepared the ourself to how to use the data.

Notebook Enviroment

- Kernel: Python 3
- 

In [3]:
!python --version

Python 3.6.3 :: Anaconda custom (64-bit)


## Question to be answered:

1. How many classes can we use?
2. How many images perclass?

## TODO

# Load Modules

In [2]:
import pandas as pd
import numpy as np
import json
import os
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

import seaborn as sns



# Commonly Shared Statics

In [13]:
data_labels_description_dir_path = '../../data/doc/labels/'

# EDA on the class-descriptions.csv

In [25]:
class_descriptions_file_path = data_labels_description_dir_path + 'class-descriptions.csv'


In [26]:
df_class_descriptions = pd.read_csv(class_descriptions_file_path,names=['LabelName', 'Description'])

In [27]:
df_class_descriptions.head(10)

,LabelName,Description
0,/m/0100nhbf,Sprenger's tulip
1,/m/0104x9kv,Vinegret
2,/m/0105jzwx,Dabu-dabu
3,/m/0105ld7g,Pistachio ice cream
4,/m/0105lxy5,Woku
5,/m/0105n86x,Pastila
6,/m/0105ts35,Burasa
7,/m/0108_09c,Summer snowflake
8,/m/01_097,Airmail
9,/m/010dmf,Isle of man tt


In [28]:
df_class_descriptions.tail(10)

,LabelName,Description
19985,/m/0zdntfj,Candy cane sorrel
19986,/m/0zg7256,Gingerbread house
19987,/m/0zghq,Moonshine
19988,/m/0zkm,Arthropod
19989,/m/0zlm,Conservation-restoration
19990,/m/0zrpfhj,Throwing
19991,/m/0zrrls2,Red dahlia
19992,/m/0zrthkd,Brine
19993,/m/0zrv5th,Lamborghini huracán
19994,/m/0zvk5,Helmet


In [29]:
df_class_descriptions.describe()

,LabelName,Description
count,19995,19995
unique,19995,19712
top,/m/0dcjfr,Trunk
freq,1,4


In [30]:
df_class_descriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19995 entries, 0 to 19994
Data columns (total 2 columns):
LabelName      19995 non-null object
Description    19995 non-null object
dtypes: object(2)
memory usage: 312.5+ KB


## EDA on the classes-trainable.txt

In [21]:
classes_trainable_file_path = data_labels_description_dir_path + 'classes-trainable.txt'



In [31]:
df_classes_trainable = pd.read_csv(classes_trainable_file_path,names=['LabelName'])

In [32]:
df_classes_trainable.head(10)

,LabelName
0,/m/0108_09c
1,/m/010dmf
2,/m/010jjr
3,/m/010l12
4,/m/010lq47b
5,/m/010ls_cv
6,/m/01127
7,/m/01_12b
8,/m/0117_25k
9,/m/0117z


In [33]:
df_classes_trainable.describe()

,LabelName
count,7186
unique,7186
top,/m/0dcjfr
freq,1


In [36]:
df_trainables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7186 entries, 0 to 7185
Data columns (total 2 columns):
LabelName      7186 non-null object
Description    7186 non-null object
dtypes: object(2)
memory usage: 168.4+ KB


## Filter out trainable classes from class descriptions

In [34]:
df_trainables = pd.merge(df_classes_trainable,
                 df_class_descriptions[['LabelName', 'Description']],
                 on='LabelName', 
                 how='inner')

In [35]:
df_trainables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7186 entries, 0 to 7185
Data columns (total 2 columns):
LabelName      7186 non-null object
Description    7186 non-null object
dtypes: object(2)
memory usage: 168.4+ KB


In [38]:
df_trainables.head()

,LabelName,Description
0,/m/0108_09c,Summer snowflake
1,/m/010dmf,Isle of man tt
2,/m/010jjr,Amusement park
3,/m/010l12,Roller coaster
4,/m/010lq47b,Witch hat
